# Milestone 1 – Data Preparation & Initial Analysis

In [2]:
# --- Load Libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Load Dataset ---
df = pd.read_csv("GlobalWeatherRepository.csv")
df.head()

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1715849100,2024-05-16 13:15,26.6,79.8,Partly Cloudy,...,8.4,26.6,1,1,04:50 AM,06:50 PM,12:12 PM,01:11 AM,Waxing Gibbous,55
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1715849100,2024-05-16 10:45,19.0,66.2,Partly cloudy,...,1.1,2.0,1,1,05:21 AM,07:54 PM,12:58 PM,02:14 AM,Waxing Gibbous,55
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1715849100,2024-05-16 09:45,23.0,73.4,Sunny,...,10.4,18.4,1,1,05:40 AM,07:50 PM,01:15 PM,02:14 AM,Waxing Gibbous,55
3,Andorra,Andorra La Vella,42.50,1.52,Europe/Andorra,1715849100,2024-05-16 10:45,6.3,43.3,Light drizzle,...,0.7,0.9,1,1,06:31 AM,09:11 PM,02:12 PM,03:31 AM,Waxing Gibbous,55
4,Angola,Luanda,-8.84,13.23,Africa/Luanda,1715849100,2024-05-16 09:45,26.0,78.8,Partly cloudy,...,183.4,262.3,5,10,06:12 AM,05:55 PM,01:17 PM,12:38 AM,Waxing Gibbous,55


# Inspect DataSet


In [16]:
df.info()
df.describe(include="all")
df.columns



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107183 entries, 0 to 107182
Data columns (total 41 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   country                       107183 non-null  object 
 1   location_name                 107183 non-null  object 
 2   latitude                      107183 non-null  float64
 3   longitude                     107183 non-null  float64
 4   timezone                      107183 non-null  object 
 5   last_updated_epoch            107183 non-null  int64  
 6   last_updated                  107183 non-null  object 
 7   temperature_celsius           107183 non-null  float64
 8   temperature_fahrenheit        107183 non-null  float64
 9   condition_text                107183 non-null  object 
 10  wind_mph                      107183 non-null  float64
 11  wind_kph                      107183 non-null  float64
 12  wind_degree                   107183 non-nul

Index(['country', 'location_name', 'latitude', 'longitude', 'timezone',
       'last_updated_epoch', 'last_updated', 'temperature_celsius',
       'temperature_fahrenheit', 'condition_text', 'wind_mph', 'wind_kph',
       'wind_degree', 'wind_direction', 'pressure_mb', 'pressure_in',
       'precip_mm', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius',
       'feels_like_fahrenheit', 'visibility_km', 'visibility_miles',
       'uv_index', 'gust_mph', 'gust_kph', 'air_quality_Carbon_Monoxide',
       'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
       'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10',
       'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'sunrise',
       'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination'],
      dtype='object')

# Identify missing values, anomalies, and data coverage

checking for null and missing values


In [17]:
missing = df.isna().sum()
missing[missing > 0]


Series([], dtype: int64)


 Check for Anomalies

In [18]:
anomalies = {
    "temperature_low": df[df["temperature_celsius"] < -80].shape[0],
    "temperature_high": df[df["temperature_celsius"] > 60].shape[0],
    "wind_negative": df[df["wind_kph"] < 0].shape[0],
    "humidity_low": df[df["humidity"] < 0].shape[0],
    "humidity_high": df[df["humidity"] > 100].shape[0],
}

anomalies



{'temperature_low': 0,
 'temperature_high': 0,
 'wind_negative': 0,
 'humidity_low': 0,
 'humidity_high': 0}

Handle Missing or Inconsistent Entries

In [19]:
# Fill missing humidity with mean
if "humidity" in df.columns:
    df["humidity"] = df["humidity"].fillna(df["humidity"].mean())



In [ ]:
# Drop rows with missing temperature values
df = df.dropna(subset=["temperature_celsius"])



Aggregation Example (Daily → Monthly)

In [21]:
# Remove Impossible values 
df = df[
    (df["temperature_celsius"] > -80) &
    (df["temperature_celsius"] < 60) &
    (df["humidity"] >= 0) &
    (df["humidity"] <= 100)
]


# Convert Units

In [22]:
# Convert wind speed mph → km/h
df["wind_kmh"] = df["wind_mph"] * 1.60934


In [23]:
# Fahrenheit → Celsius (already included but verifying)
df["temperature_celsius_calc"] = (df["temperature_fahrenheit"] - 32) * 5/9


# Normalize Values

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df["temp_norm"] = scaler.fit_transform(df[["temperature_celsius"]])
df["humidity_norm"] = scaler.fit_transform(df[["humidity"]])
df["wind_norm"] = scaler.fit_transform(df[["wind_kmh"]])


# Aggregate Daily Data → Monthly Average

First convert date column:

In [25]:
df["last_updated"] = pd.to_datetime(df["last_updated"])


Group by month and country:

In [26]:
df_monthly = (
    df.groupby([df["last_updated"].dt.to_period("M"), "country"])
      .agg({
          "temperature_celsius": "mean",
          "humidity": "mean",
          "wind_kmh": "mean"
      })
      .reset_index()
)

df_monthly.head()


,last_updated,country,temperature_celsius,humidity,wind_kmh
0,2024-05,Afghanistan,20.290850,38.058824,11.530448
1,2024-05,Albania,25.647059,57.529412,13.101921
2,2024-05,Algeria,26.117647,45.705882,22.597027
3,2024-05,Andorra,12.408497,60.882353,9.892708
4,2024-05,Angola,29.091503,69.764706,19.605548


# Save Cleaned Data & Monthly Aggregates

In [28]:
df.to_csv("cleaned_global_weather.csv", index=False)
df_monthly.to_csv("monthly_weather.csv", index=False)

